In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow_hub as hub

from sklearn.model_selection import train_test_split, StratifiedKFold

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

# Print GPU information
print("GPU Device Name:", tf.test.gpu_device_name())

In [ ]:
trn_path = '/kaggle/input/zindidata/images/images'
Train = pd.read_csv('/kaggle/input/zindidata/Train.csv')
Test = pd.read_csv('/kaggle/input/zindidata/Test.csv')

In [ ]:
unique_classes = Train['damage'].nunique()
print("Number of unique classes:", unique_classes)

# Class Distributions
class_distribution = Train['damage'].value_counts()
print("Class distributions:\n", class_distribution)

# Visualize Class Distributions
plt.figure(figsize=(8, 6))
sns.countplot(x='damage', data=Train, palette='viridis')
plt.title('Class Distribution in Categorical Column')
plt.xlabel('Classes')
plt.ylabel('Count')
plt.show()

In [ ]:
damage = Train.damage.unique()

In [ ]:
# Load pre-trained ViT model from Kaggle's model repository
vit_model_url = "https://www.kaggle.com/models/spsayakpaul/vision-transformer/frameworks/TensorFlow2/variations/vit-b32-fe/versions/1"
vit_model = hub.KerasLayer(vit_model_url, trainable=True)

In [ ]:
n_splits = 3
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=3)

In [ ]:
batch_size = 64
datagen = ImageDataGenerator(
    channel_shift_range=0.25,
    rescale=1./255,
    shear_range=0.35,
    zoom_range=0.4,
    horizontal_flip=True,
    rotation_range=25,
    brightness_range=[0.7, 1.5],
    width_shift_range=0.2,
    height_shift_range=0.2,
    validation_split=0.2
)

In [ ]:
accuracies = []

In [ ]:
all_preds = np.zeros((len(Test), len(damage)))

In [ ]:
for fold, (train_index, valid_index) in enumerate(skf.split(Train, Train['damage'])):
    print(f"Training Fold {fold + 1}...")
    
    train_df, valid_df = Train.iloc[train_index], Train.iloc[valid_index]

    train_generator = datagen.flow_from_dataframe(
        dataframe=train_df,
        directory=trn_path,
        x_col="filename",
        y_col="damage",
        target_size=(224, 224),
        batch_size=batch_size,
        class_mode="categorical",
        subset="training"
    )

    valid_generator = datagen.flow_from_dataframe(
        dataframe=valid_df,
        directory=trn_path,
        x_col="filename",
        y_col="damage",
        target_size=(224, 224),
        batch_size=batch_size,
        class_mode="categorical",
        subset="validation"
    )

    # Build the model
    model = models.Sequential([
        vit_model,
        layers.Flatten(),
        layers.Dense(512, activation='relu'),
        layers.Dense(len(damage), activation='softmax')
    ])

    # Compile the model with custom loss
    def custom_loss(y_true, y_pred):
        # Add a custom loss term to penalize misclassifications of the zero class
        zero_class_weight = 2.0  # Adjust the weight based on your dataset characteristics
        return tf.keras.losses.categorical_crossentropy(y_true, y_pred) + zero_class_weight * tf.keras.losses.binary_crossentropy(tf.math.reduce_max(y_true, axis=-1), tf.math.reduce_max(y_pred, axis=-1))

    model.compile(optimizer=optimizers.Adam(learning_rate=1e-4),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    # Train the model
    epochs = 7
    history = model.fit(train_generator, validation_data=valid_generator, epochs=epochs)
    _, accuracy = model.evaluate(valid_generator)
    accuracies.append(accuracy)
    print(accuracy)
    # Make predictions on the test set for this fold
    test_datagen = ImageDataGenerator(rescale=1./255)
    test_generator = test_datagen.flow_from_dataframe(
        dataframe=Test,
        directory=trn_path,
        x_col="filename",
        y_col=None,
        target_size=(224, 224),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False
    )

    preds = model.predict(test_generator)
    all_preds += preds / n_splits

In [ ]:
average_accuracy = np.mean(accuracies)
print("Average Validation Accuracy: {:.2f}%".format(average_accuracy * 100))

In [ ]:
submission_df = pd.DataFrame({
    'ID': Test['ID'],
    'DR': all_preds[:, 0],
    'G': all_preds[:, 1],
    'ND': all_preds[:, 2],
    'WD': all_preds[:, 3],
    'other': all_preds[:, 4]
})
submission_df.to_csv('tensorflow_submission.csv', index=False)